In [31]:
from datasets import load_dataset, get_dataset_config_names
import pandas as pd

## Data Downloading

In [161]:
config = get_dataset_config_names('Helsinki-NLP/opus-100')
data = load_dataset('Helsinki-NLP/opus-100', 'bn-en')
data['train'].to_csv("./data/train_data.csv")

Creating CSV from Arrow format: 100%|██████████| 1000/1000 [00:05<00:00, 190.12ba/s]


265553291

## Data 

In [162]:
df = pd.read_csv('/home/arafat/projects/deeplearning_crash_course/data/train_data.csv')
df['translation_dict'] = df['translation'].apply(eval)


df['bn'] = df['translation_dict'].apply(lambda x: x['bn'])
df['en'] = df['translation_dict'].apply(lambda x: x['en'])


df = df.drop('translation_dict', axis=1)
df.to_csv("./data/bangla_english_translation.csv")

In [163]:
df.loc[100:150, ['bn', 'en']].values[:, 0]

array(['চিহ্নিত অনুসন্ধানের ফোল্ডারের নাম উল্লেখ করা আবশ্যক',
       'কাফেররা যেন মনে না করে যে আমি যে , অবকাশ দান করি , তা তাদের পক ্ ষে কল ্ যাণকর । আমি তো তাদেরকে অবকাশ দেই যাতে করে তারা পাপে উন ্ নতি লাভ করতে পারে । বস ্ তুতঃ তাদের জন ্ য রয়েছে লাঞ ্ ছনাজনক শাস ্ তি ।',
       'সেদিন তাদের চক ্ রান ্ ত তাদের কোন উপকারে আসবে না এবং তারা সাহায ্ যপ ্ রাপ ্ তও হবে না ।',
       'একটা দুঃসংবাদ দিবো।',
       "আর ইহুদী ও খ ্ রীষ ্ টানরা বলে -- ''আমরা আল্লাহ্\u200cর সন্তান ও তাঁর প্রিয়পাত্র।’’ তুমি বলো -- ''তবে কেন তোমাদের অপরাধের জন্য তিনি তোমাদের শাস্তি দেন? না, যাদের তিনি সৃষ্টি করেছেন তোমরা তাদের মধ্যেকার মানুষ। তিনি যাকে ইচ্ছে করেন পরিত্রাণ করেন এবং যাকে ইচ্ছে করেন শাস্তি দেন।’’ আর আল্লাহ্\u200cরই মহাকাশমন্ডল ও পৃথিবীর সাম্রাজ্য এবং এই দুইয়ের মধ্যে যা আছে, আর তাঁরই দিকে প্রত্যাবর্তন।",
       'এটা দেখ তো।',
       'এই রসূলগণ-আমি তাদের কাউকে কারো উপর মর ্ যাদা দিয়েছি । তাদের মধ ্ যে কেউ তো হলো তারা যার সাথে আল ্ লাহ কথা বলেছেন , আর কারও মর ্ যাদা উচ ্ চতর করেছেন এবং আমি মরিয়

## Train a Tokenizer

In [142]:
import os
from typing import Literal

from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [133]:
def train_and_save_tokenizer(file_path:str, output_path:str, vocab_size:int, lang:str = Literal['bn', 'en']):
    if not os.path.exists(file_path):
        raise FileNotFoundError("File Path Does Not Exist")
    
    df = pd.read_csv(file_path)
    df = df.dropna()
    training_data = df.loc[:, lang].values
    
    tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(vocab_size=vocab_size,
                               min_frequency=2,
                               special_tokens=["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"])
    
    tokenizer.train_from_iterator(training_data, trainer)
    
    os.makedirs(output_path, exist_ok=True)
    tokenizer.save(f"./{output_path}/{lang}_tokenizer.json")
    return tokenizer

In [160]:
path = '/home/arafat/projects/deeplearning_crash_course/data/bangla_english_translation.csv'
en_tokenizer = train_and_save_tokenizer(path, 'Tokenizer', 1000, 'en')
bn_tokenizer = train_and_save_tokenizer(path, 'Tokenizer', 1000, 'bn')

In [159]:
tokenizer = Tokenizer.from_file('/home/arafat/projects/deeplearning_crash_course/Tokenizer/en_tokenizer.json')
tokenizer.enable_padding(
    pad_id=tokenizer.token_to_id("[PAD]"),
    pad_token="[PAD]",
    length=None,
    # length=50,
    direction='right'
)

sample_text = ["This is a test sentence", "This is another sentence. But the sentence is long"]
encoding = tokenizer.encode_batch(sample_text)
encoding[0].ids, encoding[1].ids

# token = torch.stack([encoding[0].ids, encoding[1].ids], padding=True)

([90, 15, 12, 0, 0, 1, 1, 1, 1, 1], [90, 15, 218, 0, 5, 98, 7, 0, 15, 278])

## DataLoader

In [68]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
class MachineTranslation(Dataset):
    def __init__(self, file_path:str, en_tokenizer_path:str, bn_tokenizer_path:str):
        super().__init__()

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File Path does not exists for {file_path}")
        if not os.path.exists(en_tokenizer_path):
            raise FileNotFoundError(f"File Path does not exists for {en_tokenizer_path}")
        if not os.path.exists(bn_tokenizer_path):
            raise FileNotFoundError(f"File Path does not exists for {bn_tokenizer_path}")
        
        self.file_path = file_path
        self.en_tokenizer = Tokenizer.from_file(en_tokenizer_path)
        self.bn_tokenizer = Tokenizer.from_file(bn_tokenizer_path)
        
        self.en_tokenizer.enable_padding(
            pad_id=self.en_tokenizer.token_to_id("[PAD]"),
            pad_token="[PAD]",
            length=None,
            direction='right'
        )
        
        self.bn_tokenizer.enable_padding(
            pad_id=self.bn_tokenizer.token_to_id("[PAD]"),
            pad_token="[PAD]",
            length=None,
            direction='right'
        )
        
        
        
    def __getitem__(self, idx):
        csv_file = pd.read_csv(self.file_path)
        sentences = csv_file.loc[idx, ['bn', 'en']].values
        bangla, english = sentences[:, 0], sentences[:, 1]
        bn_token_id = self.bn_tokenizer.encode(bangla).ids
        en_token_id = self.en_tokenizer.encode(english).ids
        return bangla, english
        
    
    def __len__(self):
        return len()